In [4]:
import json
import pandas as pd

# Обучение

In [ ]:
with open("synthetic_data.json", "r", encoding="utf-8") as f:
    synthetic_data = json.load(f)

with open("mistral.json", "r", encoding="utf-8") as f:
    mistral_data = json.load(f)

synthetic_df = pd.json_normalize(synthetic_data)
mistral_df = pd.json_normalize(mistral_data)

df = pd.concat([synthetic_df, mistral_df], ignore_index=True)

In [6]:
df.shape

(51100, 14)

In [7]:
df.head()

,clientId,organizationId,segment,role,organizations,currentMethod,mobileApp,availableMethods,claims,recommendedMethod,signatures.common.mobile,signatures.common.web,signatures.special.mobile,signatures.special.web
0,client_84066,organization_60802,Средний бизнес,Сотрудник,54,PayControl,True,"[КЭП на токене, КЭП в приложении]",0,КЭП в приложении,23,7,28,12
1,client_87803,organization_63216,Средний бизнес,Сотрудник,174,КЭП в приложении,True,"[КЭП на токене, SMS, PayControl]",0,PayControl,8,26,41,27
2,client_93021,organization_87083,Малый бизнес,Сотрудник,167,КЭП на токене,True,"[КЭП в приложении, PayControl, SMS]",0,КЭП в приложении,50,33,26,12
3,client_14274,organization_46413,Малый бизнес,ЕИО,100,КЭП на токене,False,"[PayControl, КЭП на токене, SMS]",0,PayControl,0,7,0,8
4,client_38177,organization_2531,Малый бизнес,Сотрудник,135,КЭП на токене,True,"[КЭП в приложении, КЭП на токене]",0,КЭП в приложении,18,18,5,45


In [8]:
df.recommendedMethod.value_counts()

recommendedMethod
PayControl          18637
КЭП на токене       13052
КЭП в приложении    10049
SMS                  9362
Name: count, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51100 entries, 0 to 51099
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   clientId                   51100 non-null  object
 1   organizationId             51100 non-null  object
 2   segment                    51100 non-null  object
 3   role                       51100 non-null  object
 4   organizations              51100 non-null  int64 
 5   currentMethod              51100 non-null  object
 6   mobileApp                  51100 non-null  bool  
 7   availableMethods           51100 non-null  object
 8   claims                     51100 non-null  int64 
 9   recommendedMethod          51100 non-null  object
 10  signatures.common.mobile   51100 non-null  int64 
 11  signatures.common.web      51100 non-null  int64 
 12  signatures.special.mobile  51100 non-null  int64 
 13  signatures.special.web     51100 non-null  int64 
dtypes: boo

In [10]:
df.rename(columns={
    "signatures.common.mobile": "signatures_common_mobile",
    "signatures.common.web": "signatures_common_web",
    "signatures.special.mobile": "signatures_special_mobile",  
    "signatures.special.web": "signatures_special_web"  
}, inplace=True)

In [11]:
df.head()

,clientId,organizationId,segment,role,organizations,currentMethod,mobileApp,availableMethods,claims,recommendedMethod,signatures_common_mobile,signatures_common_web,signatures_special_mobile,signatures_special_web
0,client_84066,organization_60802,Средний бизнес,Сотрудник,54,PayControl,True,"[КЭП на токене, КЭП в приложении]",0,КЭП в приложении,23,7,28,12
1,client_87803,organization_63216,Средний бизнес,Сотрудник,174,КЭП в приложении,True,"[КЭП на токене, SMS, PayControl]",0,PayControl,8,26,41,27
2,client_93021,organization_87083,Малый бизнес,Сотрудник,167,КЭП на токене,True,"[КЭП в приложении, PayControl, SMS]",0,КЭП в приложении,50,33,26,12
3,client_14274,organization_46413,Малый бизнес,ЕИО,100,КЭП на токене,False,"[PayControl, КЭП на токене, SMS]",0,PayControl,0,7,0,8
4,client_38177,organization_2531,Малый бизнес,Сотрудник,135,КЭП на токене,True,"[КЭП в приложении, КЭП на токене]",0,КЭП в приложении,18,18,5,45


In [12]:
df['is_available_cap_on_token'] = df['availableMethods'].apply(lambda x: 1 if 'КЭП на токене' in x else 0)
df['is_available_cap_in_phone'] = df['availableMethods'].apply(lambda x: 1 if 'КЭП в приложении' in x else 0)
df['is_available_sms'] = df['availableMethods'].apply(lambda x: 1 if 'SMS' in x else 0)
df['is_available_pay_control'] = df['availableMethods'].apply(lambda x: 1 if 'PayControl' in x else 0)

df.drop(["clientId", "organizationId", "availableMethods"], inplace=True, axis=1)

In [13]:
df.head()

,segment,role,organizations,currentMethod,mobileApp,claims,recommendedMethod,signatures_common_mobile,signatures_common_web,signatures_special_mobile,signatures_special_web,is_available_cap_on_token,is_available_cap_in_phone,is_available_sms,is_available_pay_control
0,Средний бизнес,Сотрудник,54,PayControl,True,0,КЭП в приложении,23,7,28,12,1,1,0,0
1,Средний бизнес,Сотрудник,174,КЭП в приложении,True,0,PayControl,8,26,41,27,1,0,1,1
2,Малый бизнес,Сотрудник,167,КЭП на токене,True,0,КЭП в приложении,50,33,26,12,0,1,1,1
3,Малый бизнес,ЕИО,100,КЭП на токене,False,0,PayControl,0,7,0,8,1,0,1,1
4,Малый бизнес,Сотрудник,135,КЭП на токене,True,0,КЭП в приложении,18,18,5,45,1,1,0,0


In [14]:
id2word = {index: label for index, label in enumerate(df["recommendedMethod"].unique())}
word2id = {label:index for index, label in id2word.items()}

In [15]:
X, y = df.drop("recommendedMethod", axis=1), df["recommendedMethod"].map(word2id)

In [16]:
X = pd.get_dummies(X)

In [18]:
y = y.astype('category')

In [19]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)

X_res, y_res = smote.fit_resample(X, y)

In [39]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

In [40]:
y_res.value_counts()

recommendedMethod
0    18637
1    18637
2    18637
3    18637
Name: count, dtype: int64

In [ ]:
from catboost import CatBoostClassifier, Pool

train_data = Pool(X_train, label=y_train)
test_data = Pool(X_test, label=y_test)

In [ ]:
# Обучаем модель CatBoost с использованием GPU
model = CatBoostClassifier(
    iterations=1000,
    random_state=42,
    verbose=200,
    task_type='GPU',  
    devices='0' 
)

In [24]:
model.fit(train_data)

Learning rate set to 0.149448
0:	learn: 1.1137903	total: 51.2ms	remaining: 51.2s
200:	learn: 0.2905893	total: 1.9s	remaining: 7.56s
400:	learn: 0.2696866	total: 3.92s	remaining: 5.85s
600:	learn: 0.2548822	total: 5.91s	remaining: 3.92s
800:	learn: 0.2421550	total: 7.88s	remaining: 1.96s
999:	learn: 0.2309484	total: 9.77s	remaining: 0us


In [29]:
model.save_model('catboost_model.bin')

In [26]:
from sklearn.metrics import classification_report

In [27]:
y_pred = model.predict(test_data)

In [28]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.89      0.86      3710
           1       0.75      0.69      0.72      3774
           2       0.84      0.89      0.86      3663
           3       0.82      0.80      0.81      3763

    accuracy                           0.81     14910
   macro avg       0.81      0.82      0.81     14910
weighted avg       0.81      0.81      0.81     14910



# RandomForest

In [360]:
from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier(
    n_estimators=500,
    max_depth=6,
    random_state=42,
    class_weight='balanced'  # Балансировка классов
)

model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=6, n_estimators=500,
                       random_state=42)

In [361]:
y_pred_rf = model_rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86      3710
           1       0.96      0.48      0.64      3774
           2       0.81      0.90      0.85      3663
           3       0.76      0.82      0.79      3763

    accuracy                           0.80     14910
   macro avg       0.82      0.80      0.78     14910
weighted avg       0.82      0.80      0.78     14910



# XGboost

In [ ]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 10.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.0/199.0 MB 7.0 MB/s eta 0:00:00:00:0100:01


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
import xgboost as xgb

In [37]:
param_grid = {
    'n_estimators': [300, 400, 500], 
    'learning_rate': [0.01, 0.05, 0.1, 0.2],  
    'max_depth': [3, 5, 6, 7],  
    'subsample': [0.6, 0.7, 0.8, 0.9],  
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9],
}

In [ ]:
model_xgb = xgb.XGBClassifier(
    n_estimators=500,
    eval_metric='mlogloss', 
    random_state=42,
    tree_method="gpu_hist"
)

grid_search = GridSearchCV(estimator=model_xgb, param_grid=param_grid, scoring='f1_weighted', cv=3, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)

best_model = grid_search.best_estimator_

In [ ]:
y_pred_xb = best_model.predict(X_test)

In [44]:
print(classification_report(y_test, y_pred_xb))

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      3710
           1       0.72      0.70      0.71      3774
           2       0.86      0.89      0.88      3663
           3       0.82      0.80      0.81      3763

    accuracy                           0.81     14910
   macro avg       0.81      0.82      0.81     14910
weighted avg       0.81      0.81      0.81     14910

